In [7]:
from jupyter_dash import JupyterDash

from dash import Dash, dcc, html, Input, Output, dash_table, no_update  
import plotly.express as px
import json
import pandas as pd
from datetime import datetime
import numpy as np
import math

#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
time_freq = 'D'
def format_timedelta(td):
    ts = td.total_seconds()
    if not math.isnan(ts):
        str_ts = str(ts)
        decimal_part = str_ts.split('.')[1]
        hours, remainder = divmod(ts, 3600)
        minutes, seconds = divmod(remainder, 60)
        return ('{}:{:02d}:{:02d}.{:012d}').format(int(hours),int(minutes),int(seconds),int(decimal_part))
    else:
        return td


#app = Dash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash(__name__) #, external_stylesheets=external_stylesheets)
with open('/home/wong/Documents/newsmf110s1.json') as f:
    data = json.load(f)

df_norm = pd.json_normalize([elem for elem in data 
                              if ("dfhmntds" in elem.keys() and
                                  isinstance(elem['dfhmntds'], list))], 
                         'dfhmntds',
                         meta=[['header', 'smfmnprn'],
                               ['header', 'dateTime'],
                               ['header', 'sysId']],
                         errors='ignore')

df_norm = df_norm.assign(dateTime = lambda x: pd.to_datetime(x['header.dateTime'],
               format="%Y-%m-%d %H:%M:%S.%f"))
    
df_norm = df_norm.assign(start = lambda x: x['start'][:-3].astype(np.datetime64))
df_norm = df_norm.assign(stop = lambda x: x['stop'][:-3].astype(np.datetime64))

df_norm = df_norm.assign(response = lambda x: x['stop'] - x['start'])

df = df_norm[['header.sysId', 'header.smfmnprn', 'dateTime',
                    'tran', #'start', 'stop', 
                    'response', 
                    'usrcput', 'scusrhwm_a',
                    'sc31chwm', 'fctotct', 'db2reqct']]
    
df = df.set_index(['header.sysId', 'header.smfmnprn','tran', 'dateTime'])

dff = df.groupby(
    ['header.sysId','header.smfmnprn','tran']).agg({ 
    'response': 'max',
    'usrcput': 'max', 
    'scusrhwm_a': 'max',
    'sc31chwm': 'max',
    'fctotct': 'max',
    'db2reqct': 'max'
    }).reset_index()

dff["id"] = dff.index
dff = dff.assign(responseStr = lambda x: x['response'].apply(format_timedelta))
columns = ['header.sysId', 'header.smfmnprn', #'header.dateTime',
                    'tran', #'start', 'stop', 
                    'responseStr', 
                    'usrcput', 'scusrhwm_a',
                    'sc31chwm', 'fctotct', 'db2reqct']
color = {"response": "#636EFA", "usrcput": "#EF553B", "scusrhwm_a": "#00CC96",
         'sc31chwm': "orange", 'fctotct': "purple", 'db2reqct': "blue"}
initial_active_cell = {"row": 0, "column": 0, "column_id": "tran", "row_id": 0}

app.layout = html.Div(
    [
        html.Div(
            [
                html.H3("CICS Transaction Performance", style={"textAlign":"center"}),
                dash_table.DataTable(
                    id="table",
                    columns=[{"name": c, "id": c} for c in columns],
                    data=dff.to_dict("records"),
                    page_size=10,
                    sort_action="native",
                    active_cell=initial_active_cell,
                ),
            ],
            style={"margin": 50},
            #className="five columns"
        ),
        html.Div(id="output-graph", ), #className="six columns"),
    ],
    className="row"
)


@app.callback(
     Output("output-graph", "children"),
     Input("table", "active_cell"),
 )
def cell_clicked(active_cell):
    if active_cell is None:
        return no_update
    
    row = active_cell["row_id"]
    transid = dff.at[row, "tran"]
    applid = dff.at[row, "header.smfmnprn"]
    lparid = dff.at[row, "header.sysId"]
    col = active_cell["column_id"]

    df_s = df_norm[(df_norm["tran"]==transid) &
                   (df_norm["header.sysId"]==lparid)  &
                   (df_norm["header.smfmnprn"]==applid)]
    
    df_s = df_s.assign(response = lambda x: x['response'] / np.timedelta64(1, 's'))
    df_s = df_s.assign(usrcput = lambda x: x['usrcput'] / np.timedelta64(1, 's'))
    y = col if col in ['usrcput', 'scusrhwm_a',
                        'sc31chwm', 'fctotct', 'db2reqct'] else "response"

    fig = px.scatter(
        df_s, x="dateTime", y=y, title=" ".join([transid, y])
    )
    fig.update_layout(title={"font_size": 20},  title_x=0.5, margin=dict(t=190, r=15, l=5, b=5))
    fig.update_traces(line=dict(color=color[y]))

    return dcc.Graph(figure=fig)
    #return y

#if __name__ == "__main__":
app.run_server(debug=True,mode='inline', use_reloader=False)